<!-- WARNING: THIS FILE WAS AUTOGENERATED! DO NOT EDIT! -->

## Business Logic Layer

This module provides the service layer for processing chat messages and handling business logic.

In [0]:
#| echo: false
#| output: asis
show_doc(Responder)

---

### Responder

>      Responder (*args, **kwargs)

*Protocol for chat responders (LLMs, APIs, etc.).*

In [0]:
#| echo: false
#| output: asis
show_doc(ErrorHandler)

---

### ErrorHandler

>      ErrorHandler (*args, **kwargs)

*Protocol for handling errors during message processing.*

In [0]:
#| echo: false
#| output: asis
show_doc(DefaultErrorHandler)

---

### DefaultErrorHandler

>      DefaultErrorHandler (include_details:bool=True)

*Default error handler implementation.*

In [0]:
#| echo: false
#| output: asis
show_doc(ChatService)

---

### ChatService

>      ChatService (responder:Union[__main__.Responder,Callable],
>                   error_handler:Optional[__main__.ErrorHandler]=None, context_
>                   provider:Optional[Callable[[pylogue.session.ChatSession],Any
>                   ]]=None)

*Service for processing chat messages with responders.*

## Example Responders

In [0]:
#| echo: false
#| output: asis
show_doc(echo_responder)

---

### echo_responder

>      echo_responder (message:str, context:Optional[Any]=None)

*Simple echo responder for testing.*

In [0]:
#| echo: false
#| output: asis
show_doc(ContextAwareResponder)

---

### ContextAwareResponder

>      ContextAwareResponder (max_history:int=5)

*Example responder that uses conversation history.*

## Test the Service Layer

In [11]:
# Test basic ChatService
from pylogue.session import ChatSession

service = ChatService(responder=echo_responder)
response = await service.process_message("Hello, world!")
print(f"Response: {response}")

Response: [Echo] You said: Hello, world!


In [12]:
# Test with session
session = ChatSession("test-session")
service = ChatService(responder=echo_responder)

assistant_msg = await service.process_session_message(session, "First message")
print(f"Session messages: {session.get_message_dicts()}")

Session messages: [{'role': 'User', 'content': 'First message', 'id': 'eb9c84e7-f898-4514-8bf9-aac2c4ece363'}, {'role': 'Assistant', 'content': '[Echo] You said: First message', 'id': '52451b28-a288-4cb5-8ec7-c139daf736f7'}]


In [13]:
# Test context-aware responder
def context_provider(session: ChatSession):
    """Provide message history as context."""
    return session.get_messages()


session = ChatSession("context-test")
responder = ContextAwareResponder()
service = ChatService(responder=responder, context_provider=context_provider)

await service.process_session_message(session, "First")
await service.process_session_message(session, "Second")
await service.process_session_message(session, "Third")

print(f"Final messages: {session.get_message_dicts()}")

Final messages: [{'role': 'User', 'content': 'First', 'id': 'a9b83b5a-9c29-46d3-a3c3-179ca1d77b7f'}, {'role': 'Assistant', 'content': "I see we've exchanged 1 messages. You just said: First", 'id': '555f171f-fbfa-4b2c-b6e3-0e754093c006'}, {'role': 'User', 'content': 'Second', 'id': 'b1bb1984-8154-4cfb-8372-67c73315467d'}, {'role': 'Assistant', 'content': "I see we've exchanged 3 messages. You just said: Second", 'id': '471f8465-d3b2-46f5-8fb1-e14cef42a030'}, {'role': 'User', 'content': 'Third', 'id': '9cd1d761-fa34-4eca-aadf-acffe180a238'}, {'role': 'Assistant', 'content': "I see we've exchanged 5 messages. You just said: Third", 'id': 'e47b7fba-92ab-4fbf-829f-9e86d8afb7a5'}]


In [14]:
# Test error handling
def failing_responder(message: str, context=None):
    raise ValueError("Something went wrong!")


service = ChatService(responder=failing_responder)
response = await service.process_message("This will fail")
print(f"Error response: {response}")

Error response: Error processing message: ValueError: Something went wrong!
